In [23]:
import time
import sys
import numpy as np
from collections import Counter
from nltk.corpus import stopwords

# Let's tweak our network from before to model these phenomena
class SentimentNetwork:
    def __init__(self, reviews,labels,min_count = 10,polarity_cutoff = 0.1,hidden_nodes = 10, learning_rate = 0.1):
       
        np.random.seed(1)
    
        self.pre_process_data(reviews, polarity_cutoff, min_count)
        
        self.init_network(len(self.review_vocab),hidden_nodes, 1, learning_rate)
        
        
    def pre_process_data(self,reviews, polarity_cutoff,min_count):
        
        positive_counts = Counter()
        negative_counts = Counter()
        total_counts = Counter()
        stopset = set(stopwords.words('portuguese'))

        for i in range(len(reviews)):
            if(labels[i] == 'POSITIVE'):
                for word in reviews[i].split(" "):
                    if word not in stopset:
                        positive_counts[word] += 1
                        total_counts[word] += 1
            else:
                for word in reviews[i].split(" "):
                    if word not in stopset:
                        negative_counts[word] += 1
                        total_counts[word] += 1

        pos_neg_ratios = Counter()

        for term,cnt in list(total_counts.most_common()):
            if(cnt >= 5):
                pos_neg_ratio = positive_counts[term] / float(negative_counts[term]+1)
                pos_neg_ratios[term] = pos_neg_ratio

        for word,ratio in pos_neg_ratios.most_common():
            if(ratio > 1):
                pos_neg_ratios[word] = np.log(ratio)
            else:
                pos_neg_ratios[word] = -np.log((1 / (ratio + 0.01)))
        
        review_vocab = set()
        for review in reviews:
            for word in review.split(" "):
                if(total_counts[word] > min_count):
                    if(word in pos_neg_ratios.keys()):
                        if((pos_neg_ratios[word] >= polarity_cutoff) or (pos_neg_ratios[word] <= -polarity_cutoff)):
                            review_vocab.add(word)
                    else:
                        review_vocab.add(word)
        self.review_vocab = list(review_vocab)
        
        label_vocab = set()
        for label in labels:
            label_vocab.add(label)
        
        self.label_vocab = list(label_vocab)
        
        self.review_vocab_size = len(self.review_vocab)
        self.label_vocab_size = len(self.label_vocab)
        
        self.word2index = {}
        for i, word in enumerate(self.review_vocab):
            self.word2index[word] = i
        
        self.label2index = {}
        for i, label in enumerate(self.label_vocab):
            self.label2index[label] = i
         
        
    def init_network(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        # Set number of nodes in input, hidden and output layers.
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes

        # Initialize weights
        self.weights_0_1 = np.zeros((self.input_nodes,self.hidden_nodes))
    
        self.weights_1_2 = np.random.normal(0.0, self.output_nodes**-0.5, 
                                                (self.hidden_nodes, self.output_nodes))
        
        self.learning_rate = learning_rate
        
        self.layer_0 = np.zeros((1,input_nodes))
        self.layer_1 = np.zeros((1,hidden_nodes))
        
    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))
    
    
    def sigmoid_output_2_derivative(self,output):
        return output * (1 - output)
    
    def update_input_layer(self,review):

        # clear out previous state, reset the layer to be all 0s
        self.layer_0 *= 0
        for word in review.split(" "):
            self.layer_0[0][self.word2index[word]] = 1

    def get_target_for_label(self,label):
        if(label == 'POSITIVE'):
            return 1
        else:
            return 0
        
    def train(self, training_reviews_raw, training_labels):
        
        training_reviews = list()
        for review in training_reviews_raw:
            indices = set()
            for word in review.split(" "):
                if(word in self.word2index.keys()):
                    indices.add(self.word2index[word])
            training_reviews.append(list(indices))
        
        assert(len(training_reviews) == len(training_labels))
        
        correct_so_far = 0
        
        start = time.time()
        
        for i in range(len(training_reviews)):
            
            review = training_reviews[i]
            label = training_labels[i]
            
            #### Implement the forward pass here ####
            ### Forward pass ###

            # Input Layer

            # Hidden layer
#             layer_1 = self.layer_0.dot(self.weights_0_1)
            self.layer_1 *= 0
            for index in review:
                self.layer_1 += self.weights_0_1[index]
            
            # Output layer
            layer_2 = self.sigmoid(self.layer_1.dot(self.weights_1_2))

            #### Implement the backward pass here ####
            ### Backward pass ###

            # Output error
            layer_2_error = layer_2 - self.get_target_for_label(label) # Output layer error is the difference between desired target and actual output.
            layer_2_delta = layer_2_error * self.sigmoid_output_2_derivative(layer_2)

            # Backpropagated error
            layer_1_error = layer_2_delta.dot(self.weights_1_2.T) # errors propagated to the hidden layer
            layer_1_delta = layer_1_error # hidden layer gradients - no nonlinearity so it's the same as the error

            # Update the weights
            self.weights_1_2 -= self.layer_1.T.dot(layer_2_delta) * self.learning_rate # update hidden-to-output weights with gradient descent step
            
            for index in review:
                self.weights_0_1[index] -= layer_1_delta[0] * self.learning_rate # update input-to-hidden weights with gradient descent step

            if(layer_2 > 0.5 and label == 'POSITIVE'):
                correct_so_far += 1
            if(layer_2 <= 0.5 and label == 'NEGATIVE'):
                correct_so_far += 1
            
            reviews_per_second = i / float(time.time() - start)
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(training_reviews)))[:4] + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] + " #Correct:" + str(correct_so_far) + " #Trained:" + str(i+1) + " Training Accuracy:" + str(correct_so_far * 100 / float(i+1))[:4] + "%")
        
    
    def test(self, testing_reviews, testing_labels):
        
        correct = 0
        
        start = time.time()
        
        for i in range(len(testing_reviews)):
            pred = self.run(testing_reviews[i])
            if(pred == testing_labels[i]):
                correct += 1
            
            reviews_per_second = i / float(time.time() - start)
            
            sys.stdout.write("\rProgress:" + str(100 * i/float(len(testing_reviews)))[:4] \
                             + "% Speed(reviews/sec):" + str(reviews_per_second)[0:5] \
                            + "% #Correct:" + str(correct) + " #Tested:" + str(i+1) + " Testing Accuracy:" + str(correct * 100 / float(i+1))[:4] + "%")
    
    def run(self, review):
        
        # Input Layer


        # Hidden layer
        self.layer_1 *= 0
        unique_indices = set()
        for word in review.lower().split(" "):
            if word in self.word2index.keys():
                unique_indices.add(self.word2index[word])
        for index in unique_indices:
            self.layer_1 += self.weights_0_1[index]
        
        # Output layer
        layer_2 = self.sigmoid(self.layer_1.dot(self.weights_1_2))
        
        if(layer_2[0] > 0.5):
            return "POSITIVE"
        else:
            return "NEGATIVE"
        
    def runDebug(self, review):
        
        # Input Layer


        # Hidden layer
        self.layer_1 *= 0
        unique_indices = set()
        for word in preproccessText(review).split(" "):
            if word in self.word2index.keys():
                unique_indices.add(self.word2index[word])
        for index in unique_indices:
            self.layer_1 += self.weights_0_1[index]
        
        # Output layer
        layer_2 = self.sigmoid(self.layer_1.dot(self.weights_1_2))

        if(layer_2[0] > 0.5):
            return("POSITIVE",layer_2[0][0])
        else:
            return("NEGATIVE",layer_2[0][0])
    def printWords(self):
        print(self.word2index.keys())
        print(self.review_vocab_size)
                    

In [24]:
import re
from unidecode import unidecode

mentions_pattern = re.compile("(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)")
url_pattern = re.compile(
r'((?:<a href[^>]+>)|(?:<a href="))?'
r'((?:https?):(?:(?://)|(?:\\\\))+'
r"(?:[\w\d:#@%/;$()~_?\+\-=\\\.&](?:#!)?)*)",
flags=re.IGNORECASE)
special_chars_pattern = re.compile("[^ a-zA-Z#]")
shortword = re.compile(r'\W*\b\w{1}\b')

def preproccessText(text):
    aux = re.sub(mentions_pattern, " ", text)
    aux = aux.lower()
    aux = re.sub(url_pattern, " ", aux)
    aux = re.sub('\d'," ", aux)
    aux = unidecode(aux)
    aux = re.sub(special_chars_pattern, "",aux)
    aux = re.sub(shortword,"",aux)
    aux = aux.strip(' \t\n\r')
    
    aux.encode("ascii")
    return aux

In [25]:
reviews=[]
labels=[]

import csv

with open('db/all_class_tweets.csv', 'r') as csvfile:  # this will close the file automatically.
    reader = csv.reader(csvfile)
    
    for row in reader:
        aux = preproccessText(row[0])
        reviews.append(aux)
        labels.append(row[1])
        


In [26]:
mlp = SentimentNetwork(reviews[:-400],labels[:-400],min_count=1,polarity_cutoff=0.05,learning_rate=0.01)

In [27]:
mlp.train(reviews[:-400],labels[:-400])

Progress:99.7% Speed(reviews/sec):3821. #Correct:309 #Trained:420 Training Accuracy:73.5%

In [28]:
mlp.test(reviews[-400:],labels[-400:])

Progress:99.7% Speed(reviews/sec):5592.% #Correct:285 #Tested:400 Testing Accuracy:71.2%

In [29]:
mlp.runDebug("melhora essa internet")

('NEGATIVE', 0.43550888495696605)

In [30]:
mlp.runDebug("🙃")

('NEGATIVE', 0.38717956015971955)

In [ ]:
import socket
from http.server import BaseHTTPRequestHandler, HTTPServer
import time
import simplejson
import json

hostName = ""
hostPort = 8081

class MyServer(BaseHTTPRequestHandler):

    #GET is for clients geting the predi
    def do_GET(self):
        self.send_response(200)
        self.wfile.write(bytes("<p>You accessed path: %s</p>" % self.path, "utf-8"))

    #POST is for submitting data.
    def do_POST(self):
        
        content_length = int(self.headers['Content-Length']) # <--- Gets the size of data
        post_data = self.rfile.read(content_length) # <--- Gets the data itself
        data = simplejson.loads(post_data)
        
        classify = mlp.runDebug(preproccessText(str(data["text"])))
        
        self.send_response(200)
        self.end_headers()
        json_string = json.dumps({"classificationPredict": classify[0], "nnResult": classify[1]})
        self.wfile.write(bytes(json_string, "utf-8"))


myServer = HTTPServer((hostName, hostPort), MyServer)
print(time.asctime(), "Server Starts - %s:%s" % (hostName, hostPort))

myServer.serve_forever()

myServer.server_close()
print(time.asctime(), "Server Stops - %s:%s" % (hostName, hostPort))

Tue Aug 15 23:55:59 2017 Server Starts - :8081


127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/2017 23:56:03] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [15/Aug/20

127.0.0.1 - - [16/Aug/2017 00:06:59] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:07:09] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:07:19] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:07:29] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:07:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:08:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:08:28] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:08:54] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:08:54] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:08:54] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:08:54] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:08:54] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:08:54] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:08:54] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:08:54] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:08:54] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/20

127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/20

127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:18] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/20

...

127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:37] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/20

127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:39] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/20

127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/20

127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:43] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/20

127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/2017 00:10:45] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [16/Aug/20

In [31]:
mlp.printWords()

dict_keys(['', 'infelizmente', 'ouvir', 'cortou', 'dias', 'pagar', 'melhores', 'porem', 'mundo', 'vivo', 'casa', 'pedi', 'propaganda', 'conseguiu', 'senha', 'ontem', 'nunca', 'pegando', 'cidadao', 'trabalhar', 'muita', 'dm', 'devo', 'voce', 'resolver', 'nenhuma', 'rsrs', 'sendo', 'mb', 'disso', 'acordar', 'playlist', 'velocidade', 'nada', 'falo', 'momento', 'hj', 'funcionando', 'pede', 'reais', 'usar', 'ruim', 'porra', 'tudo', 'desde', 'conheca', 'destaques', 'pq', 'chega', 'cancelar', 'madrugada', 'alem', 'chato', 'vcs', 'sabia', 'semana', 'rio', 'tipo', 'fiz', 'toda', 'ainda', 'gb', 'aguento', 'anos', 'despreparados', 'presente', 'colabora', 'todo', 'falta', 'mal', 'videu', 'atendentes', 'botei', 'ligou', 'existe', 'vir', 'nesse', 'boa', 'sair', 'cliente', 'conexao', 'nenhum', 'vai', 'resolvi', 'coloco', 'certo', 'fiquei', 'campeas', 'rt', '#sextou', 'galera', 'absurdo', 'completamente', 'causa', 'fixo', 'suporte', 'msm', 'agr', 'betalab', 'telefonica', 'abril', 'meia', 'in', 'dizer'

In [32]:
mlp.runDebug("@Vivoemrede Hihi ❤❤,nunca me abandone meu amor")

('POSITIVE', 0.50128456050715919)